# Using EP2BRICK

Quick introduction to using EP2BRICK, to extract a BRICK model from an EnergyPlus IDF.

In [ ]:
import sys
sys.path.append('..')
import ep2brick

Instantiate an Extractor object. This is the main class for the ep2brick tool.

The `load` method uses Opyplus to parse an IDF and find relationships between components (e.g., which components are part of a given Unitary HVAC System component).

The `walkElements` method will parse all the elements of the IDF you have just loaded, and it will convert them to `Component` objects. These objects contain raw data on all the objects extracted from the IDF.

The `connectComponents` method will search for fluid loops and connect them using the BRICK predicate "feeds" (and its inverse "isFedBy")

In [ ]:
ext = ep2brick.Extractor()
opm = ext.load('../tests/resources/school.idf')
test = ext.walkElements(opm)
ext.connectComponents()

Some EnergyPlus components cannot be directly translated to BRICK components. Examples include NodeLists, Splitters, Mixers, etc.

The `cleanupPredicates` will find any component that cannot be matched to a BRICK component and delete it, while at the same time fixing the predicates for all related objects. Without this step, some predicates might point to non-existent objects or objects with no BRICK type.

In [ ]:
ext.cleanupPredicates()
#LOOP DUE TO WATER:CONNECTIONS AND WATER:EQUIPMENT

In [ ]:
for subj in ext.components.values():
    if subj.btype != "":
        print(f"{subj.name}, a {subj.btype}")
        for predicate in subj.bpredicates.keys():
            for obj in subj.bpredicates[predicate]:
                try:
                    print(f"\t {predicate} {obj.name}, a {obj.btype}")
                except:
                    print(f"\t {predicate} {obj} !!STRING!!")

Finally, the `createGraph` method will generate an empty graph and populate it with the information we just extracted from an IDF.

The `saveGraph` method serializes the graph and outputs a Turtle (.ttl) file, which by default has the same name as the original IDF - except for the file extension.

In [ ]:
ext.createGraph()
ext.saveGraph()